### Administrative arrests assesement and analysis / Gibran Mena-Aguilar

## Task 1 Search for Mahmoud Khalil

Is Mahmoud Khalil in the dataset? The highest probability is that he is not. As the evidence I gathered in the exploratory analysis of the database documentation points to. 

The most important piece of knowledge to understand this that this database does not represent the full universe of immigration-related arrests, only "administrative arrests" performed by Enforcement Removal Operations (ERO) within Immigration and Customs Enforcement (ICE), as reported by the professor and legal expert Austin Kocher (linked in the documentation of the data)

The key question to report on is if Khalil's notice to appear order (NTA) performed on March the 8th was issued by ERO or by Homeland Security Investigations (HSI), which is the case since he was targeted not on violations of immigration policies but on the supposed grounds of posing "potentially serious adverse foreign policy consequences for the United States" according to US Secretary of State Marco Rubio.

Administrative arrests should be linked to administrative warrants but there are exceptions, for isntance, arrests can an are being performed by ERO without warrants if they argue they have reason to believe the person in question is either entering the country illegaly or is already in the country and likely to escape before a warrant can be obtained. Therefore, some arrests with no warrant issued could be included in the database but only the ones enacted by ERO.

Still, drilling down in the database is a useful excercise to see what records within say.

Documention of the dataset states that "located" and "non-custodial arrest" are most likely to be detentions not at a jail or prison.AP and NYTimes record the date of arrest on Saturday March the 8th in his Columbia university owned apartement, which The Guardian states is blocks away from the main campus.

But we need to take into consideration that data is not complete, 21% is missing in the column "State Apprehension" but only 2% is missing from the column "Area of Reponsibility" (AOR), which can be much more closer to ground truth. Age has a 100% of records, therefore I filtered for all individuals born in 1995 detained on March the 8th, a day before and a day after, to explore the results. 

There are 58 records with this characteristics, none of them with a missing value under AOR. Then there are 2 records in the New York City AOR, one with a criminal criminal conviction departed to El Salvador and arrested on March the 7th, which excludes him from the scope and the other with 2 criminal pending charges and a Salvadorean nationality, where the criminal charges are the most telling feature to exclude him to be a match. 

My recommendation would be to file a FOIA request or get a hold of the HSI arrests database. 

## Task 2

The team wants to understand whether the "Apprehension Site Landmark" column could provide clues about potential partnerships between local and state law enforcement and ICE.

Use this column to help identify which jail or prison individuals have been taken to, where available. Add one column to the dataset indicating the name of the facility, and another showing the county where that facility is located.

---

This is not the most accurate way to obtain information on what jails, facilities and counties individuales have been take to, since this information is available in the rest of the tables of the dataset, detentions and detainers. However, within the dataset, "Apprehension Site Landmark" is indeed the most granular variable relevant to the county of apprehension and jails or prisons, as AOR can cover more than one state and hence not very tied to counties at all. 

String analysis based on string lenght within the column shows most common arrangements that are not part of the name of the county before the word "county" are "Fugitive Operations" or "Fugitive operations on street" or "At large arrest", "All areas inside", "CAP", standing for Custody Alternatives Program and a combination of one to three letters, a white space before and after a dash, stating the abbreviation of a state, all of which I excluded to then extract the counties from the column. 

I created a function that detects a parameter, in this case a string, that can be changed from "county" to "jail" and "prison", or "facility". It excludes the said patterns and then retrieves all words before each of these two words. Then I extracted the string match to one of two columns: either "county_name" or "facility_name". 

The objective is to look for in the column for commons ways to refer to prisons and match those patterns in the Apprehension Site Landamark. 

With this method, though, there is only the possibility to retrieve 44% of the counties.

For the facilities, I included in my function a way to detect the most commons facility patterns: the words "center", "correctional", "penitentiary", "removal" as the end words of such terms as "service processing center", for instance. 

I then added a different approach: download the public database of the 3000+ counties where I searched for any matches of them in the column "Apprehension Site Landmark".

Questions:

where are the raids most succesful, meaning number of NTAs vs arrests, which profiles lead to removals and which not, denoting possible sucessful law strategies on the ground from migrant organizations, and/or unsuccesful law strategies from the team
Criminal history 

What age race, ethnicity and citizenship is targeted. What are tha proxies for class there? What is the gender component? 
Males are being detained, this means that women are left to sustain their families and communities, how are they doing it on the ground, what are their resources and strategies., This means a new women-empowered but also left on the shoulders of now single or away from partners mothers . This is yet another instance of family separation?

What legal status is being most removed?

removal methods, destination countries, besides El Salvador

Why in the world is someone supposedly detained in Mexico??? 
















In [ ]:
import re
from fuzzywuzzy import process

def extract_fuzzy_cleaned_phrases(text, basis_word="county", choices=None, threshold=90):
    """
    For each instance of the basis_word (e.g., 'county'), remove unwanted phrases/patterns before it,
    and extract the cleaned phrase before and including the basis_word.
    Then, fuzzy match the cleaned phrase against a list of choices (e.g., county names or facility names).
    Returns the best fuzzy match if above threshold, else None.
    """
    if not isinstance(text, str):
        return None

    # Unwanted phrases/patterns (case-insensitive)
    unwanted_phrases = [
        r"fugitive operations",
        r"fugitive operations on street",
        r"at large arrest",
        r"all areas inside",
        r"cap",
        r"\b[a-zA-Z]{1,3}\s-\s"
    ]
    unwanted_pattern = re.compile("|".join(unwanted_phrases), re.IGNORECASE)

    results = []
    for match in re.finditer(rf"(.*?\b{basis_word}\b)", text, re.IGNORECASE):
        phrase = match.group(1)
        cleaned = unwanted_pattern.sub("", phrase)
        cleaned = cleaned.strip()
        if cleaned:
            results.append(cleaned + f" {basis_word}")

    # Fuzzy match against choices if provided
    if choices and results:
        matches = []
        for cleaned_phrase in results:
            match, score = process.extractOne(cleaned_phrase, choices)
            if score >= threshold:
                matches.append(match)
        if matches:
            return "; ".join(matches)
        else:
            return None
    elif results:
        return "; ".join(results)
    else:
        return None

# Example usage:
# counties_list = ["Harris County", "Kings County", "Los Angeles County", ...]  # your list of counties
# df_arrests["fuzzy_county"] = df_arrests["Apprehension Site Landmark"].apply(
#     lambda x: extract_fuzzy_cleaned_phrases(x, basis_word="county", choices=counties_list, threshold=85)
# )



**Note:**  
- You need to install `fuzzywuzzy` and `python-Levenshtein` for best performance:  
  `pip install fuzzywuzzy python-Levenshtein`
- Replace `counties_list` with your list of valid county or facility names.  
- Adjust the `threshold` parameter for stricter or looser matching.